# [PUBLIC] Analyse OpenCL Caffe profiling

We analyse profiling **performance** (execution time) of OpenCL Caffe with the following:
- CNN **models** (net + weights):
  - GoogleNet;
  - AlexNet;
  - SqueezeNet 1.0;
  - SqueezeNet 1.1;

- OpenCL BLAS **libraries**:
  - CLBlast dev (> 0.10.0).

## Includes

### Standard

In [ ]:
import os
import sys
import json

### Scientific

In [ ]:
import IPython as ip
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mp

In [ ]:
print 'IPython version: %s' % ip.__version__
print 'NumPy version: %s' % np.__version__
print 'SciPy version: %s' % sp.__version__
print 'Pandas version: %s' % pd.__version__
print 'Matplotlib version: %s' % mp.__version__

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
# import scipy.stats as st

### Collective Knowledge

In [ ]:
import ck.kernel as ck
print 'CK version: %s' % ck.__version__

In [ ]:
r=ck.access({'action':'show', 'module_uoa':'env', 'tags':'tool,opencl,dvdt,prof'})
if r['return']>0:
    print ("Error: %s" % r['error'])
    exit(1)
# Get path the first returned environment entry.
dvdt_prof_dir=r['lst'][0]['meta']['env']['CK_ENV_TOOL_DVDT_PROF']
dvdt_prof_src_python=os.path.join(dvdt_prof_dir,'src','python')
sys.path.append(dvdt_prof_src_python)
import prof_wrangler as pw
pw.test()

## Access experimental results

In [ ]:
def get_experimental_results(tags):
    r=ck.access({'action':'search', 'module_uoa':'experiment', 'tags':tags})
    if r['return']>0:
        print ("Error: %s" % r['error'])
        exit(1)
    # FIXME: For now, assume a single entry per the given tags.
    r=ck.access({'action':'list_points', 'module_uoa':'experiment', 'data_uoa': r['lst'][0]['data_uoa']})
    if r['return']>0:
        print ("Error: %s" % r['error'])
        exit(1)
    
    results = {}
    for point in r['points']:
        with open(os.path.join(r['path'], 'ckp-%s.0001.json' % point)) as point_file:
            point_data_raw = json.load(point_file)
            run_info_list = [
                characteristics['run']
                for characteristics in point_data_raw['characteristics_list']
                if characteristics['run']['run_success'] == 'yes'
            ]
            # Select characteristics of interest. TODO: simplify.
            point_data_dict = {
                'time_fw_ms' : [ run_info.get('time_fw_ms',0)      for run_info in run_info_list ],
                'dvdt_prof'  : [ run_info.get('dvdt_prof',{})      for run_info in run_info_list ],
                'per_layer'  : [ run_info.get('per_layer_info',[]) for run_info in run_info_list ]
            }
            batch_size = point_data_raw['choices']['env']['CK_CAFFE_BATCH_SIZE']
            results[batch_size] = point_data_dict
    return results

In [ ]:
def get_min_time_fw_per_image(results):
    time_fw_ms = { k : v['time_fw_ms'] for (k,v) in results.iteritems()}
    df = pd.DataFrame(data=time_fw_ms)
    df.index.name = 'repetition'
    df.columns.name = 'batch size'
    return (df.describe().ix['min'] / range(1,len(time_fw_ms)+1)).min()

### AlexNet

In [ ]:
alexnet_clblast_tags = 'dvdt-prof,caffe,opencl-clblast,bvlc-alexnet'
alexnet_clblast_results = get_experimental_results(alexnet_clblast_tags)
alexnet_clblast_min_time_fw_per_image = get_min_time_fw_per_image(alexnet_clblast_results)
alexnet_clblast_min_time_fw_per_image

### SqueezeNet 1.0

In [ ]:
squeezenet_1_0_clblast_tags = 'dvdt-prof,caffe,opencl-clblast,deepscale-squeezenet-1.0'
squeezenet_1_0_clblast_results = get_experimental_results(squeezenet_1_0_clblast_tags)
squeezenet_1_0_clblast_min_time_fw_per_image = get_min_time_fw_per_image(squeezenet_1_0_clblast_results)
squeezenet_1_0_clblast_min_time_fw_per_image

### SqueezeNet 1.1

In [ ]:
squeezenet_1_1_clblast_tags = 'dvdt-prof,caffe,opencl-clblast,deepscale-squeezenet-1.1'
squeezenet_1_1_clblast_results = get_experimental_results(squeezenet_1_1_clblast_tags)
squeezenet_1_1_clblast_min_time_fw_per_image = get_min_time_fw_per_image(squeezenet_1_1_clblast_results)
squeezenet_1_1_clblast_min_time_fw_per_image

### GoogleNet

In [ ]:
googlenet_clblast_tags = 'dvdt-prof,caffe,opencl-clblast,bvlc-googlenet'
googlenet_clblast_results = get_experimental_results(googlenet_clblast_tags)
googlenet_clblast_min_time_fw_per_image = get_min_time_fw_per_image(googlenet_clblast_results)
googlenet_clblast_min_time_fw_per_image

## Execution time - data frame

In [ ]:
data = {
    'dvdt-prof' : {
        'AlexNet'      : alexnet_clblast_min_time_fw_per_image,
        'SqueezeNet 1.0' : squeezenet_1_0_clblast_min_time_fw_per_image,
        'SqueezeNet 1.1' : squeezenet_1_1_clblast_min_time_fw_per_image,
        'GoogleNet'      : googlenet_clblast_min_time_fw_per_image,
    },
}
df = pd.DataFrame(data)
df

## Execution time - plot

In [ ]:
ymax = 500 #df.max()*1.05
df.T \
    .plot(title='Execution time per image (ms)',
          kind='bar', rot=0, ylim=[0,ymax], figsize=[12, 6], grid=True, legend=True, colormap=cm.autumn)

# Execution profile

In [ ]:
# Preferred time unit ('ns', 'us', 'ms', 's').
unit = 'ms'

In [ ]:
alexnet_clblast_batchsize1_trace0 = alexnet_clblast_results[1]['dvdt_prof'][0]
trace = alexnet_clblast_batchsize1_trace0
trace = pw.index_calls(trace)

In [ ]:
# Partial trace only containing kernel enqueues.
kernel_enqueues = pw.filter_calls(trace, ['clEnqueueNDRangeKernel'])

In [ ]:
# # Use a simple helper that works for any enqueue. Scale by 1e-6 to get ms.
# df_kernel_enqueues = pw.df_enqueues_ns(kernel_enqueues) * 1e-6
# df_kernel_enqueues

In [ ]:
# Use a more sophisticated helper that works only for kernel enqueues.
df_kernel_enqueues = pw.df_kernel_enqueues(kernel_enqueues, unit)
df_kernel_enqueues

In [ ]:
# Cumulative kernel execution time according to timestamps.
df_kernel_enqueues[['t1 - t0 (%s)' % unit]].sum()

In [ ]:
# Cumulative kernel execution time according to profiling.
df_kernel_enqueues[['p3 - p2 (%s)' % unit]].sum()

In [ ]:
# Show kernel enqueues that take at least 1 ms to execute and with at least 5% of overhead in profiling.
df_kernel_enqueues[
    (df_kernel_enqueues['t1 - t0 (ms)'] > 1.00) & \
    (df_kernel_enqueues['t1 - t0 (ms)'] > 1.05 * df_kernel_enqueues['p3 - p0 (ms)'])
]

In [ ]:
df_kernel_enqueues_cumulative_time_num = pw.df_kernel_enqueues_cumulative_time_num(df_kernel_enqueues, unit)
df_kernel_enqueues_cumulative_time_num[
    df_kernel_enqueues_cumulative_time_num['** Execution time (%) **'] > 0.2